In [1]:
import pandas as pd
import re
import os

# Ruta base de salida
ruta_base = "/Users/joccerrillo/Downloads"

# 1. Leer el CSV con codificación Latino Multilingüe (cp858)
df = pd.read_csv(r"/Users/joccerrillo/Downloads/PRECIOS2.CSV", encoding="cp858")

# --- Función para extraer cantidad y unidad ---
def extraer_y_normalizar(presentacion):
    if not isinstance(presentacion, str):
        return None

    texto = presentacion.upper()
    match = re.search(r'(\d+(?:\.\d+)?)\s*(KG|GR|LT|ML)', texto)

    if match:
        cantidad = float(match.group(1))
        unidad = match.group(2)

        # Normalización
        if unidad == "KG":
            cantidad *= 1000
            unidad = "GR"
        elif unidad == "LT":
            cantidad *= 1000
            unidad = "ML"

        # Crear cadena como "1000ML" o "850GR"
        return f"{int(cantidad) if cantidad.is_integer() else cantidad}{unidad}"

    return None

# Aplicar la función
df["cantidad_normalizada"] = df["presentacion"].apply(extraer_y_normalizar)

# --- Separar cantidad y unidad ---
df[["cantidad_valor", "unidad"]] = df["cantidad_normalizada"].str.extract(r"(\d+(?:\.\d+)?)([A-Z]+)")
df["cantidad_valor"] = pd.to_numeric(df["cantidad_valor"], errors="coerce")

# --- Calcular el precio por unidad ---
df["precio_por_unidad"] = df["precio"] / df["cantidad_valor"]

# --- Agrupar por producto y unidad ---
df_promedios = (
    df.groupby(["producto", "unidad"], as_index=False)
      .agg(precio_promedio_por_unidad=("precio_por_unidad", "mean"))
)

# --- Normalizar a precio por 100 g / 100 ml ---
df_promedios["precio_promedio_por_100"] = df_promedios.apply(
    lambda x: x["precio_promedio_por_unidad"] * 100 if x["unidad"] in ["GR", "ML"] else None,
    axis=1
)

# --- Redondear ---
df_promedios[["precio_promedio_por_unidad", "precio_promedio_por_100"]] = df_promedios[
    ["precio_promedio_por_unidad", "precio_promedio_por_100"]
].round(4)

# --- Mostrar en consola ---
print("\n✅ Precio promedio por producto (normalizado a 100 g / 100 ml):")
print(df_promedios.head(10))

# --- 1️⃣ Exportar valores únicos a un solo archivo Excel ---
ruta_excel = os.path.join(ruta_base, "valores_unicos2.xlsx")

with pd.ExcelWriter(ruta_excel) as writer:
    for columna in df.columns:
        valores_unicos = df[columna].dropna().unique()
        df_unicos = pd.DataFrame(valores_unicos, columns=[columna])
        df_unicos.to_excel(writer, sheet_name=columna[:31], index=False)
    print(f"✅ Archivo Excel generado: {ruta_excel}")

# --- 2️⃣ Exportar resumen de precios promedio a CSV ---
ruta_csv = os.path.join(ruta_base, "precio_promedio.csv")
df_promedios.to_csv(ruta_csv, index=False, encoding="utf-8-sig")

print(f"✅ Archivo CSV generado: {ruta_csv}")


✅ Precio promedio por producto (normalizado a 100 g / 100 ml):
          producto unidad  precio_promedio_por_unidad  precio_promedio_por_100
0           ACEITE     ML                      0.0491                   4.9119
1  ACEITE DE OLIVA     ML                      0.3002                  30.0234
2         ACEITUNA     GR                      0.3081                  30.8079
3     AGUA CON GAS     ML                      0.0575                   5.7535
4     AGUA SIN GAS     ML                      0.0128                   1.2782
5         AGUACATE     GR                      0.0804                   8.0367
6              AJO     GR                      0.3899                  38.9852
7         AJONJOLÍ     GR                      0.1757                  17.5685
8        ALCACHOFA     GR                      0.1747                  17.4743
9        ALCAPARRA     GR                      0.3042                  30.4246
✅ Archivo Excel generado: /Users/joccerrillo/Downloads/valores_unic

## Fusión con la base de datos BAM

In [ ]:
import pandas as pd
import re
import os

# --- 1. Diccionario de equivalencias ---
diccionario_mapeo = {
    "ACEITE DE OLIVA": "ACEITE, DE OLIVA",
    "AGUA CON GAS": "AGUA MINERAL",
    "AGUA SIN GAS": "AGUA EMBOTELLADA O GARRAFON",
    "AJONJOLÍ": "AJONJOLI",
    "ALIMENTO PREPARADO PARA NIÑOS": "ALIMENTO PARA BEBE",
    "ALIMENTOS PREPARADOS": "NULL",
    "ATÚN": "ATUN Y SARDINA (PROMEDIO)",
    "BANDERA": "None",
    "BARRAS DE AVENA": "BARRA DE AVENA QUAKER TODOS LOS SABORES",
    "BARRITA DE SURIMI O BARRITA DE SURIMI CANGRE": "None",
    "BASA": "PESCADO FRESCO, PROMEDIO",
    "BEBIDAS HIDRATANTES": "BEBIDA REHIDRATANTE (GATORADE, POWERADE)",
    "BLANCO DEL NILO": "PESCADO FRESCO, TILAPIA",
    "BRANDY": "NULL",
    "BRÓCOLI": "BROCOLI",
    "CAFÉ SOLUBLE": "CAFE DILUIDO SIN AZUCAR, INDUSTRIALIZADO",
    "CAFÉ TOSTADO Y MOLIDO": "CAFE EXPRESSO, PREPARACION ESTANDARIZADA",
    "CAMARÓN": "CAMARON CRUDO",
    "CARNE CERDO": "CERDO, CARNE COCIDA",
    "CARNE PAVO": "PAVO O GUAJOLOTE (PROMEDIO)",
    "CARNE POLLO": "POLLO, PROMEDIO",
    "CARNE RES": "RES, CARNE COCIDA",
    "CARNE TERNERA": "TERNERA, CARNE EN CANAL",
    "CAZÓN": "PESCADO FRESCO, CAZON",
    "CHILES": "CHILE JALAPEÑO EN ESCABECHE (ENTERO), INDUSTRIALIZADO",
    "CHOCOLATE EN TABLILLAS": "CHOCOLATE CON AZUCAR (ABUELITA, IBARRA, ETC)",
    "CHUCUMITE": "PESCADO FRESCO, ROBALO",
    "CHÍA": "SEMILLA DE CHIA",
    "CHÍCHARO": "CHICHARO",
    "CHÍCHAROS": "CHICHARO",
    "CHÍCHAROS EN LATA": "CHICHARO, INDUSTRIALIZADO",
    "COCTEL DE FRUTAS EN ALMÍBAR": "ALMIBAR DE FRUTAS, INDUSTRIALIZADO",
    "COMBINADO PRIMAVERA": "VERDURAS MIXTAS, INDUSTRIALIZADAS",
    "CONCENTRADO DE POLLO": "CONSOME (KNORR SUIZA, ROSA BLANCA, ETC.)",
    "CREMA BATIDA": "CREMA CHANTILLY",
    "CURVINA": "PESCADO FRESCO, PROMEDIO",
    "CÚRCUMA": "CURCUMA (YUQUILLA)",
    "DURAZNOS EN ALMÍBAR": "DURAZNO EN ALMIBAR (SIN SEMILLA), INDUSTRIALIZADO",
    "EMPANIZADORES": "PAN MOLIDO",
    "FLORETES DE BRÓCOLI": "BROCOLI",
    "FÓRMULA LÁCTEA": "None",
    "GALLETAS DULCES": "GALLETA DULCE (TIPO MARIA, ANIMALITO)",
    "GALLETAS POPULARES": "GALLETA DULCE (TIPO MARIA, ANIMALITO)",
    "GALLETAS SALADAS": "GALLETA SALADA, NO GRASOSA (SALADITAS, KRAKER BRAN, PREMIUM, HABANERAS)",
    "GRASA COMESTIBLE": "MANTECA, VEGETAL PROMEDIO",
    "GUANÁBANA": "GUANABANA CIMARRONA",
    "GUARNICIÓN DE VERDURAS": "VERDURAS COCIDAS CON SAL",
    "HARINA HOT CAKES": "HARINA PARA HOT CAKES",
    "HUACHINANGO": "PESCADO FRESCO, PROMEDIO",
    "JARABE P/PREPARAR BEBIDAS": "JARABES DE SABORES (CONCENTRADOS PARA AGUA DE SABOR)",
    "JUREL": "PESCADO FRESCO, PROMEDIO",
    "JÍCAMA": "JICAMA",
    "LECHE EN POLVO": "LECHE ENTERA, EN POLVO",
    "LECHE EVAPORADA": "LECHE ENTERA, EVAPORADA",
    "LECHE PASTEURIZADA": "LECHE LICONSA PROMEDIO",
    "LECHE ULTRAPASTEURIZADA": "LECHE, SANTA CLARA",
    "LOBINA": "PESCADO FRESCO, ROBALO",
    "MANTECA DE CERDO": "MANTECA, ANIMAL PROMEDIO",
    "MAÍZ PALOMERO": "MAIZ PALOMERO",
    "MAÍZ POZOLERO": "MAIZ PRECOCIDO (PARA POZOLE)",
    "MELÓN": "MELON PROMEDIO",
    "MERLUZA": "PESCADO FRESCO, PROMEDIO",
    "MEZCLA CALIFORNIA": "VERDURAS COMBINADAS (PARA SOPA O CALDO), CRUDAS",
    "MEZCLA CAMPESINA": "VERDURAS MIXTAS, INDUSTRIALIZADAS",
    "MIEL DE ABEJA": "MIEL, DE ABEJA",
    "MOLE ROJO EN PASTA": "MOLE (PASTA PARA PREPARAR), INDUSTRIALIZADO",
    "ORÉGANO": "OREGANO",
    "PAPAS FRITAS Y SIMILARES": "PAPAS EN HOJUELAS (TODAS MARCAS: SABRITAS, BARCEL, ETC.)",
    "PASA (UVA PASA)": "PASAS",
    "PASTEL PIMIENTO": "QUESO PIMIENTO",
    "PASTELILLOS Y PAN DULCE EMPAQUETADO": "PAN DULCE INDUSTRIALIZADO (EXCEPTO FRITOS COMO DONAS Y CHURROS)",
    "PERÓN": "PERON",
    "PETO": "NULL",
    "PIÑA EN ALMÍBAR": "PIÑA EN ALMIBAR, INDUSTRIALIZADA",
    "PLÁTANO": "PLATANO (PROMEDIO)",
    "POLVO P/HORNEAR": "POLVO PARA HORNEAR",
    "POLVO P/PREPARAR BEBIDAS": "NULL",
    "POLVO P/PREPARAR BEBIDAS SABOR CHOCOLATE": "CHOCOLATE EN POLVO ENRIQUECIDO (NESQUIK, CALCTOSE)",
    "PRODUCTO LÁCTEO": "NULL",
    "PURÉ DE TOMATE": "PURE DE TOMATE, INDUSTRIALIZADO",
    "PÁMPANO": "PESCADO FRESCO, PAMPANO",
    "QUESO AMERICANO": "QUESO PASTEURIZADO AMERICANO (FORTIFICADO CON VITAMINA D)",
    "QUESO CANASTO": "QUESO PANELA O FRESCO, DE VACA",
    "QUESO COTIJA": "QUESO AÑEJO TIPO COTIJA",
    "QUESO DOBLE CREMA": "QUESO PHILADELPHIA",
    "QUESO SIERRA": "QUESO (PROMEDIO)",
    "ROBALITO": "PESCADO FRESCO, ROBALO",
    "RUBIA": "None",
    "SAL DE MAR": "SAL",
    "SAL MOLIDA DE MESA": "SAL",
    "SALMÓN": "SALMON FRESCO",
    "SALSA PICANTE Y SIMILARES": "SALSA ROJA MEXICANA, INDUSTRIALIZADA",
    "SANDÍA": "SANDIA (OVALADA)",
    "SEMILLA DE GIRASOL": "SEMILLA SECA DE GIRASOL CON CASCARA",
    "SUSTITUTO DE AZÚCAR": "ESTEVIA (SPLENDA NATURALS)",
    "TORTILLA DE HARINA DE TRIGO": "TORTILLA, DE HARINA DE TRIGO (FORTIFICADA)",
    "TORTILLA DE MAÍZ": "TORTILLA NIXTAMALIZADA (PROMEDIO)",
    "TOCINO AHUMADO": "TOCINO DE PUERCO",
    "TOSTADAS DE MAÍZ": "TOSTADA O TORTILLA FRITA, DE MAIZ INDUSTRIALIZADA (TIPO MILPA REAL)",
    "TÉ": "TE (PROMEDIO)",
    "TÉ PREPARADO": "TE (INFUSION SIN AZUCAR), PREPARACION ESTANDARIZADA",
    "YOGHURT": "YOGURT NATURAL (LECHE ENTERA)",
    "FÓRMULA LÁCTEA": "FORMULA LACTEA ENTERA (TIPO NUTRILECHE), FLUIDA"
}

# --- 2. Leer los archivos ---
ruta_base = "/Users/joccerrillo/Downloads/Maestría en Ciencias e Ingenieria de la Computación/Clase IA/Proyecto Final"

df_precios = pd.read_csv(os.path.join(ruta_base, "precio_promedio.csv"), encoding="utf-8-sig")
df_bam = pd.read_csv(os.path.join(ruta_base, "bam.csv"), encoding="utf-8-sig")

# --- 3. Normalizar texto ---
df_precios["producto_norm"] = df_precios["producto"].str.upper().str.strip()
df_bam["nombre_norm"] = df_bam["nombre_del_alimento"].str.upper().str.strip()

# --- 4. Columnas nutricionales ---
columnas_nutri = [
    "energ_kcal","carbohydrt","lipid_tot","protein","fiber_td","calcium","iron","ironhem","ironnohem",
    "zinc","vit_c","thiamin","riboflavin","niacin","panto_acid","vit_b6","folic_acid","food_folate",
    "folate_dfe","vit_b12","vit_a_rae","vit_e","vit_d_iu","vit_k","fa_sat","fa_mono","fa_poly","chole"
]

# --- 5. Promediar valores nutricionales usando diccionario ---
rows = []

for producto in df_precios["producto_norm"].unique():
    # Determinar palabra de búsqueda según el diccionario
    clave = diccionario_mapeo.get(producto, producto)
    if clave and clave.upper() not in ["NULL", "NONE"]:
        patron_seguro = re.escape(clave.upper())
        coincidencias = df_bam[df_bam["nombre_norm"].str.contains(patron_seguro, na=False, regex=True)]
    else:
        coincidencias = pd.DataFrame()  # evitar falsas coincidencias
    
    if not coincidencias.empty:
        promedios = coincidencias[columnas_nutri].mean(numeric_only=True)
        promedios["producto_norm"] = producto
        promedios["clave_usada"] = clave
        promedios["coincidencias_en_bam"] = len(coincidencias)
        rows.append(promedios)
    else:
        row_vacia = pd.Series({col: None for col in columnas_nutri})
        row_vacia["producto_norm"] = producto
        row_vacia["clave_usada"] = clave
        row_vacia["coincidencias_en_bam"] = 0
        rows.append(row_vacia)

# --- 6. Crear dataframe con promedios ---
df_bam_prom = pd.DataFrame(rows)

# --- 7. Join con precios ---
df_final = pd.merge(df_precios, df_bam_prom, on="producto_norm", how="left")

# --- 8. Exportar ---
ruta_csv = os.path.join(ruta_base, "precios_nutricion_combinado.csv")
df_final.to_csv(ruta_csv, index=False, encoding="utf-8-sig")

print("✅ Archivo combinado generado correctamente:")
print(f"➡ {ruta_csv}")

✅ Archivo combinado generado correctamente:
➡ /Users/joccerrillo/Downloads/Maestría en Ciencias e Ingenieria de la Computación/Clase IA/Proyecto Final/precios_nutricion_combinado.csv
